In [1]:
import pandas as pd
import numpy as np

import rpy2 #python interface for R
import rpy2.robjects as robjects #high level interface with R for "pythonic" implementation
from rpy2.robjects.packages import importr #for importing R packages in python
import rpy2.robjects.packages as rpackages #for downloading R libraries in python

In [2]:
rpy2.__version__

'3.4.5'

### Importing R packages in Python

In [3]:
# importing R libraries in python
base = importr('base')
# utils = importr('utils')

In [4]:
#installing R packages in python
utils = rpackages.importr('utils') #utils is used to install pacakges

# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list

<rpy2.rinterface_lib.sexp.NULLType object at 0x0000011ED489E8C0> [RTYPES.NILSXP]

### Installing R packages in Python from Python
- To install R packages in python you have to run it as an administrator - so if you run jupyter lab from bash then run bash as an admin

### Alternative
- If you have a local installation of R on your machine then download R packages directly by using R and then import in python.

In [18]:
# install packages
packnames = ('ggplot2', 'caret', 'glmnet')

# R vector of strings
from rpy2.robjects.vectors import StrVector

#selectively install only packages which need to be installed
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
# if len(names_to_install) > 0:
#     utils.install_packages(StrVector(names_to_install))

### Using named R objects in Python

In [6]:
pi = robjects.r['pi']
pi[0]

3.141592653589793

### Calling Functions in R

In [7]:
#creat a R function in the R global environment using robjects.r
robjects.r('''
    # create a function 'f'
    f <- function(r, verbose=FALSE){
        if(verbose){
            cat("Calling f().\n")
        }
        2 * pi * r
    }
    # call the function 'f' with argument value 3
    f(3)
''')

18.849556


In [8]:
r_f = robjects.globalenv['f']
print(r_f.r_repr())

function (r, verbose = FALSE) 
{
    if (verbose) {
        cat("Calling f().\n")
    }
    2 * pi * r
}


In [9]:
f = robjects.r['f']
result = f(3)
print(result)

[1] 18.84956



In [10]:
### another exxample on calling builtin functions in R
rsum = robjects.r['sum']
rvec = robjects.IntVector([1,2,3]) #define a R vector of integers

res = rsum(rvec) #call the sum function on the vector defined above

print(res)

[1] 6



### Working with R packages
- when working with symbols in R - translation needs to be done using a dict before it can be used in python
- make sure the translation is not masking other symbols used in R

In [11]:
#example translating r objects to another name before import
d = {'package.dependencies': 'package_dot_dependencies',
     'package_dependencies': 'package_uscore_dependencies'}
tools = importr('tools', robject_translations = d)

In [12]:
def get_key(val):
    for key, value in dict(tools.__dict__).items():
         if val == value:
                return {key: value}
    return "key doesn't exist"

get_key(tools.__dict__['package_uscore_dependencies']) #shows that now the function exist in the environment with the translated name

{'package_uscore_dependencies': <rpy2.robjects.functions.DocumentedSTFunction object at 0x0000011EDC4C2D00> [RTYPES.CLOSXP]
 R classes: ('function',)}

### Importing R code as a package
- Wrap R code inside a python string and import as a python module

In [20]:
from trans_lasso import trans_lasso
from rpy2.robjects.packages import SignatureTranslatedAnonymousPackage

In [21]:
methods = trans_lasso()

In [22]:
trans_lasso_methods = SignatureTranslatedAnonymousPackage(methods, "trans_lasso_methods")

In [27]:
trans_lasso_methods.__dict__ #the imported functions get . translated to _

{'__name__': 'trans_lasso_methods',
 '__doc__': None,
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '_env': <rpy2.robjects.environments.Environment object at 0x0000011EDDF54080> [RTYPES.ENVSXP]
 R classes: ('environment',)
 n items: 7,
 '__rname__': 'trans_lasso_methods',
 '_translation': {},
 '_rpy2r': {'agg_fun': 'agg.fun',
  'ind_set': 'ind.set',
  'las_kA': 'las.kA',
  'mse_fun': 'mse.fun',
  'rep_col': 'rep.col',
  'Trans_lasso': 'Trans.lasso',
  'Trans_lasso_sp': 'Trans.lasso.sp'},
 '_exported_names': {'Trans_lasso',
  'Trans_lasso_sp',
  'agg_fun',
  'ind_set',
  'las_kA',
  'mse_fun',
  'rep_col'},
 '_symbol_r2python': <function rpy2.robjects.packages_utils.default_symbol_r2python(rname: str) -> str>,
 '_symbol_resolve': <function rpy2.robjects.packages_utils.default_symbol_resolve(symbol_mapping)>,
 'agg_fun': <rpy2.robjects.functions.SignatureTranslatedFunction object at 0x0000011EDDF7BE80> [RTYPES.CLOSXP]
 R classes: ('function',),
 'ind_set': <rpy2.robjects

In [26]:
print(trans_lasso_methods.Trans_lasso)

function (X, y, n.vec, I.til, l1 = T) 
{
    M = length(n.vec) - 1
    X0.til <- X[I.til, ]
    y0.til <- y[I.til]
    X <- X[-I.til, ]
    y <- y[-I.til]
    Rhat <- rep(0, M + 1)
    p <- ncol(X)
    n.vec[1] <- n.vec[1] - length(I.til)
    ind.1 <- ind.set(n.vec, 1)
    for (k in 2:(M + 1)) {
        ind.k <- ind.set(n.vec, k)
        Xty.k <- t(X[ind.k, ]) %*% y[ind.k]/n.vec[k] - t(X[ind.1, 
            ]) %*% y[ind.1]/n.vec[1]
        margin.T <- sort(abs(Xty.k), decreasing = T)[1:round(n.vec[1]/3)]
        Rhat[k] <- sum(margin.T^2)
    }
    Tset <- list()
    k0 = 0
    kk.list <- unique(rank(Rhat[-1]))
    for (kk in 1:length(kk.list)) {
        Tset[[k0 + kk]] <- which(rank(Rhat[-1]) <= kk.list[kk])
    }
    k0 = length(Tset)
    Tset <- unique(Tset)
    beta.T <- list()
    init.re <- las.kA(X = X, y = y, A0 = NULL, n.vec = n.vec, 
        l1 = l1)
    beta.T[[1]] <- init.re$beta.kA
    beta.pool.T <- beta.T
    for (kk in 1:length(Tset)) {
        T.k <- Tset[[kk]]
       